# Game Theory 

In [26]:
%reset -f
# ! pip install nashpy gambit hypothesis
# ! apt-get install python-tk

import tensorflow as tf
print('TF version:' + tf.__version__)
import tensorflow as tf

# force float
from __future__ import division

    
import numpy as np


TF version:1.0.1


In [27]:
import nash
from gambit import *
from Tkinter import Tk, Button
from tkFont import Font
from copy import deepcopy

import unittest
import numpy as np

from hypothesis import given
from hypothesis.extra.numpy import arrays
from hypothesis.strategies import integers

# Refs:
https://github.com/oyamad/game_theory_models/blob/master/game_theory.ipynb
https://www3.nd.edu/~apilking/Math10120/Lectures/Topic%2029.pdf
https://github.com/Nikoleta-v3/Game-Theory-and-Python

# Definition equilibrium
An equilibrium point of a game where both players may use mixed strategies is a pair of mixed strategies such that neither player has any incentive to unilaterally change to another mixed strategy.


When searching for optimal mixed strategies for both players, we assume a number of things:
-  The pay-off matrix is known to both players.
-  Whatever mixed strategy is played by either player can be deduced by the opponent by observation.
-  Both players strive to maximize their expected payoff (note that in a zero sum game, the expected payoffs of the players add to zero, therefore maximizing an expected payoff for one player is equivalent to minimizing the expected payoff of the other player).
-  Whatever mixed strategy a player chooses, their opponent will choose the best counterstrategy

## Minimax Theorem: John Von Neumann 

For every zero sum game, there is a number ν (for value) and particular mixed strategies (optimal mixed strategies) for both players such that
1. The expected payoff to the row player will be at least ν if the row player plays his or her particular mixed strategy, no matter what mixed strategy the column player plays.
2. The expected payoff to the row player will be at most ν if the column player plays his or her particular mixed strategy, no matter what strategy the row player chooses. the number ν is called the value of the game and represents the expected advantage to the row player (a disadvantage if ν is negative).


# Calculating the utility of a pair of strategies

## Example :
Consider the situation where the payoff matrix for the row player is given by:

$$\begin{split}A =
\begin{pmatrix}
     7 & 3 \\
     2 &  5    
\end{pmatrix}\end{split}$$


$$\begin{split}A =
\begin{pmatrix}
     0 & -1 &  1\\
     1 &  0 & -1\\
    -1 &  1 &  0
\end{pmatrix}\end{split}$$

If the row player played Scissors (the 3rd strategy) and the column player played Paper (the 2nd strategy) then the row player gets: $A_{32}=1$ because Scissors cuts Paper.

A mathematical approach to representing a strategy is to consider a vector of the size: the number of strategies. For example $\sigma_r=(0, 0, 1)$ and  $\sigma_r=(0, 1, 0)$ is the row strategy where the row player always plays their third strategy. Similarly σc=(0,1,0)σc=(0,1,0) is the strategy for the column player where they always play their second strategy.

When we represent strategies like this we can get the utility to the row player using the following linear algebraic expression:

$$\sigma_r A \sigma_c^T$$


In [93]:
A = np.array([[0, -1, 1], [1, 0, -1], [-1, 1, 0]])
B = - A

# Pure
x=np.array([[0, 0, 1]])
y=np.array([[0, 1, 0]])

# print (A)

def calcUtilityNP(x,A,y):
    tp1=np.dot(x,A)    
    tp1=np.dot(tp1,y.T)
    
    tp2=np.dot(x,-A)    
    tp2=np.dot(tp2,y.T)            
    return np.array([tp1,tp2]).T


print (calcUtilityNP(x,A,y))

x=np.array([[0, 0, 1]])
# The column player decides to play Rock and Paper “randomly”.
y=np.array([[0.5, .5, 0]])
print (calcUtilityNP(x,A,y))

x=np.array([[0, 0.5, 0.5]])
# The column player decides to play Rock and Paper “randomly”.
y=np.array([[0.5, .5, 0]])
print (calcUtilityNP(x,A,y))

# Compare with http://nashpy.readthedocs.io/en/stable/tutorial/index.html#introduction-to-game-theory

[[[ 1 -1]]]
[[[ 0.  0.]]]
[[[ 0.25 -0.25]]]


# Strictly determined games

In [103]:
def isStrictlyDetermined(A):    
#     print (A)
    minimax=np.max (np.min (A, axis=0))
    maximin=np.min (np.max (A, axis=1))
    
    print ('Equal?=' + str (minimax==maximin))
    
    return minimax, maximin 
    
print (isStrictlyDetermined(A))    

Equal?=False
(-1, 1)


## Calculating Optimal mixed strategies 
the value of the game for a 2 × 2 payoff matrix with no Saddle Point CAN BE FOUND algebraically

See http://www.math.mcgill.ca/vetta/CS764.dir/Lemke.pdf
https://www.rand.org/content/dam/rand/pubs/reports/2006/R1538.pdf


The expected value of R’s payof:

$$\begin{split}=X'*A*Y \end{split}$$

Which Results in Optimal mixed strategies:

$$ H = 
  \begin{matrix}\begin{pmatrix}x & y\end{pmatrix}\\\mbox{}\end{matrix}
  \begin{pmatrix} a & b \\ c & d \end{pmatrix} 
  \begin{pmatrix} x \\ y \end{pmatrix}
$$




# RPS game

In [24]:
A = np.array([[0,  -1, 1],
               [1,  0,  -1 ],
               [-1, 1, 0 ]])


In [ ]:
# https://github.com/s3rvac/lemke-howson/blob/master/src/lh.py

In [94]:
# Verify with http://banach.lse.ac.uk/

# A = np.array([[1, 2], [3, 1]])

matching_pennies = nash.Game(A)
matching_pennies.lemke_howson(initial_dropped_label=0)

(array([ 0.33333333,  0.33333333,  0.33333333]),
 array([ 0.33333333,  0.33333333,  0.33333333]))

In [4]:
"""This module provides a representation of matrices and some common
functions operating on matrices."""

def dotPower(m,n):
    tp = np.array(m)
    for i in range(1,n):
        tp=tp.dot(tp)
    return tp        
    
# tp= tp.dot(tp)
# print tp

tp_power= dotPower(tp,3)
print tp_power

[[ 0.40234375  0.19921875  0.3984375 ]
 [ 0.3984375   0.203125    0.3984375 ]
 [ 0.3984375   0.19921875  0.40234375]]


# TF

In [5]:
import numpy as np

def dotTF(m):
    #define session
    s=tf.Session()
    s.run(tf.global_variables_initializer())

    with s:                
        xmat_1_plh = tf.placeholder(dtype=m.dtype, shape=m.shape)
        xmat_2_plh = tf.placeholder(dtype=m.dtype, shape=m.shape)                  
        r1 = tf.matmul(xmat_1_plh, xmat_2_plh)                
        qq1 = s.run(r1, feed_dict={xmat_1_plh: m , xmat_2_plh: m})
    return qq1 


def dotPowerTF(m,n,s):    
    for i in range(1,n):
        m=dotTF(m,s)                     
    return m            

mat_1=tp

m_tf=mat_1    
m_tf=dotTF(m_tf)
# m_tf=dotTF(m_tf)
# m_tf=dotTF(m_tf)
# m_tf=dotTF(m_tf)

print m_tf
# print m_tf
# b=dotPowerTF(mat_1,5,s)
# print b



[[ 0.4375  0.1875  0.375 ]
 [ 0.375   0.25    0.375 ]
 [ 0.375   0.1875  0.4375]]


In [ ]:
def calcUtilityTF(x,A,y):    
    s=tf.Session()
    s.run(tf.global_variables_initializer())
    with s:
        x_plh = tf.constant(x, dtype=tf.float64)
        y_plh = tf.constant(y, dtype=tf.float64)
        
        A_plh = tf.constant(A, dtype=tf.float64)        
#         H=((tf.reduce_sum(tf.multiply(tf.expand_dims(x_plh,-1), A_plh), axis=0)).eval())
#         H=((tf.reduce_sum(tf.multiply(tf.expand_dims(x_plh,-1), A_plh), axis=0)).eval())                
#         print((tf.reduce_sum(tf.multiply(x_plh, tf.transpose(A_plh)), axis=1)).eval())

        # Note tf.multiply is equivalent to "*"
        x_plh=((tf.reduce_sum(tf.expand_dims(x_plh,-1) * A_plh, axis=0)).eval())
        print((tf.reduce_sum(y_plh * tf.transpose(A_plh), axis=1)).eval())

        
#         r1=tf.matmul(tf.expand_dims(x_plh,0), A_plh)        
#         qq1 = s.run(r1, feed_dict={x_plh: x , A_plh: A})
#     return qq1 

# a_tf=A
# a_tf=calcUtilityTF(x,a_tf,y)